## Preparing Data

As in the previous notebooks, we'll prepare the data. 

Unlike the previous notebook with the FastText model, we no longer explicitly need to create the bi-grams and append them to the end of the sentence.

As convolutional layers expect the batch dimension to be first we can tell TorchText to return the data already permuted using the `batch_first = True` argument on the field.

In [58]:
import torch
from torchtext import data
from torchtext import datasets
import random
import numpy as np
import pandas as pd 
import codecs
import pickle,re
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


# load previously saved word vectors
with open('w2v.pkl','rb') as f:
    w2v=pickle.load(f)
w2idx = {token: token_index for token_index, token in enumerate(w2v.index2word)}


## CUSTOM DATA 
with open('pos.txt', 'r', encoding = "ISO-8859-1") as f:
    pos = [(1,p) for p in f.readlines()]
with open('neg.txt', 'r', encoding = "ISO-8859-1") as f:
    neg = [(0,n) for n in f.readlines()]

def clean_str(string, TREC=False):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Every dataset is lower cased except for TREC
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'`]", " ", string)
    string = re.sub(r"\'(s|ve|re|d|ll)", r" \'\1", string)
    string = re.sub(r"n\'t", r" n\'t", string)
    string = re.sub(r"([,!)(?]|\s{2,})", r" \1 ", string)
    return string.strip() if TREC else string.strip().lower()

mrdata = pos + neg
random.shuffle(mrdata)
writer = codecs.open('data.txt', 'w', encoding='utf-8')
for line in mrdata:
    cleaned=clean_str(line[1])
    writer.write(f"{line[0]}\t{cleaned}\n")
writer.close()


 # define fields
TEXT = data.Field(sequential=True, 
                    use_vocab=True,
                    tokenize=lambda x: x.split(' '),
                    lower=False,
                    batch_first=True,
                    fix_length=20)
LABEL = data.Field(sequential=False, unk_token=None)

dataset = data.TabularDataset('data.txt', fields=[('label', LABEL), ('text', TEXT)], format='csv', 
                              csv_reader_params={'delimiter': '\t'}, skip_header=True)

train_data, test_data=dataset.split(split_ratio=0.9,random_state=random.seed(SEED))
train_data, valid_data = train_data.split(split_ratio=7./9, random_state = random.seed(SEED))

TEXT.build_vocab(train_data,max_size=15000)
LABEL.build_vocab(train_data)
TEXT.vocab.set_vectors(w2idx, torch.from_numpy(w2v.vectors).float().to(device), w2v.vector_size)

BATCH_SIZE=64
# create data iterators
train_iterator = data.Iterator(train_data, batch_size=BATCH_SIZE, train=True, device=device, sort_key=lambda x: len(x.text), sort_within_batch=False)
valid_iterator = data.Iterator(valid_data, batch_size=BATCH_SIZE, train=False, device=device, sort_key=lambda x: len(x.text), sort_within_batch=False)
test_iterator = data.Iterator(test_data, batch_size=BATCH_SIZE, train=False, device=device, sort_key=lambda x: len(x.text), sort_within_batch=False)
print(f"train {len(train_iterator)} valid {len(valid_iterator)} test_iterator {len(test_iterator)}")

train 117 valid 34 test_iterator 17


In [59]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.conv_0 = nn.Conv2d(in_channels = 1, 
                                out_channels = n_filters, 
                                kernel_size = (filter_sizes[0], embedding_dim))
        
        self.conv_1 = nn.Conv2d(in_channels = 1, 
                                out_channels = n_filters, 
                                kernel_size = (filter_sizes[1], embedding_dim))
        
        self.conv_2 = nn.Conv2d(in_channels = 1, 
                                out_channels = n_filters, 
                                kernel_size = (filter_sizes[2], embedding_dim))
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved_0 = F.relu(self.conv_0(embedded).squeeze(3))
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [88]:

class CNN1d(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout, pad_idx):        
        super().__init__()        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = embedding_dim, 
                                              out_channels = n_filters, 
                                              kernel_size = fs)
                                    for fs in filter_sizes
                                    ])        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)        
        self.dropout = nn.Dropout(dropout)        
    def forward(self, text):        
        #text = [batch size, sent len]        
        embedded = self.embedding(text)                
        #embedded = [batch size, sent len, emb dim]        
        embedded = embedded.permute(0, 2, 1)        
        #embedded = [batch size, emb dim, sent len]        
        conved = [F.relu(conv(embedded)) for conv in self.convs]            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]        
        #pooled_n = [batch size, n_filters]        
        cat = self.dropout(torch.cat(pooled, dim = 1))        
        #cat = [batch size, n_filters * len(filter_sizes)]            
        return self.fc(cat)

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

def train(model, iterator, optimizer, criterion):    
    epoch_loss = 0
    epoch_acc = 0    
    model.train() 
    for batch in iterator:
        # print(batch.text)
        optimizer.zero_grad()        
        predictions = model(batch.text).squeeze(1)        
        loss = criterion(predictions, batch.label.float())        
        acc = binary_accuracy(predictions, batch.label)        
        loss.backward()
        optimizer.step()        
        epoch_loss += loss.item()
        epoch_acc += acc.item()    
        print(epoch_loss)    
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):    
    epoch_loss = 0
    epoch_acc = 0    
    model.eval()    
    with torch.no_grad():    
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)            
            loss = criterion(predictions, batch.label.float())            
            acc = binary_accuracy(predictions, batch.label.float())
            epoch_loss += loss.item()
            epoch_acc += acc.item()        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [89]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
                
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
                
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

We can also implement the above model using 1-dimensional convolutional layers, where the embedding dimension is the "depth" of the filter and the number of tokens in the sentence is the width.

We'll run our tests in this notebook using the 2-dimensional convolutional model, but leave the implementation for the 1-dimensional model below for anyone interested. 

In [90]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

# Initialize a model here...
model = CNN1d(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]



In [91]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,861,201 trainable parameters


In [92]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0177,  0.1787,  0.1357,  ..., -0.4316,  0.0913,  0.0762],
        [ 0.2100,  0.2598,  0.0311,  ..., -0.2695,  0.0771,  0.2197],
        [ 0.0457, -0.2119, -0.1396,  ..., -0.0481,  0.1377, -0.0320]])

In [93]:

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

## Train the Model

Training is the same as before. We initialize the optimizer, loss function (criterion) and place the model and criterion on the GPU (if available)

In [94]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

We implement the function to calculate accuracy...

In [95]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

We define a function for training our model...

**Note**: as we are using dropout again, we must remember to use `model.train()` to ensure the dropout is "turned on" while training.

In [104]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        

        predictions = model(batch.text).squeeze(1).long()
        loss = criterion(predictions, batch.label)
        print(loss)
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

We define a function for testing our model...

**Note**: again, as we are now using dropout, we must remember to use `model.eval()` to ensure the dropout is "turned off" while evaluating.

In [105]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Let's define our function to tell us how long epochs take.

In [106]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Finally, we train our model...

In [107]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    print(epoch)
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

0
tensor([ 0.0107,  0.0465,  0.0361,  0.0982,  0.0881,  0.0503,  0.0676,  0.0618,
         0.0091,  0.0695, -0.0521,  0.1292,  0.0544,  0.0338,  0.0586,  0.0637,
         0.0979,  0.0181,  0.0081,  0.1013,  0.0350,  0.1647, -0.0132,  0.0660,
         0.0774, -0.0035,  0.0403,  0.0715, -0.0191, -0.0214, -0.0295,  0.1836,
        -0.0189,  0.0696,  0.1049,  0.1650, -0.0356, -0.0805,  0.0450,  0.0431,
         0.0284,  0.0576,  0.0410, -0.0694,  0.1298,  0.0634,  0.0621, -0.0159,
        -0.0465, -0.0119, -0.0138,  0.1069,  0.1443, -0.0375,  0.0375,  0.0152,
         0.0562, -0.0201, -0.0138, -0.0038,  0.0131,  0.0497,  0.0515,  0.1655],
       device='cuda:0', grad_fn=<SqueezeBackward1>)


RuntimeError: result type Float can't be cast to the desired output type Long

We get test results comparable to the previous 2 models!

In [41]:
model.load_state_dict(torch.load('tut4-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

RuntimeError: Error(s) in loading state_dict for CNN:
	Missing key(s) in state_dict: "conv_0.weight", "conv_0.bias", "conv_1.weight", "conv_1.bias", "conv_2.weight", "conv_2.bias". 
	Unexpected key(s) in state_dict: "convs.0.weight", "convs.0.bias", "convs.1.weight", "convs.1.bias", "convs.2.weight", "convs.2.bias". 
	size mismatch for embedding.weight: copying a param with shape torch.Size([25002, 100]) from checkpoint, the shape in current model is torch.Size([15002, 300]).

## User Input

And again, as a sanity check we can check some input sentences

**Note**: As mentioned in the implementation details, the input sentence has to be at least as long as the largest filter height used. We modify our `predict_sentiment` function to also accept a minimum length argument. If the tokenized input sentence is less than `min_len` tokens, we append padding tokens (`<pad>`) to make it `min_len` tokens.

In [20]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence, min_len = 5):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

An example negative review...

In [21]:
predict_sentiment(model, "This film is terrible")

0.11022213101387024

An example positive review...

In [22]:
predict_sentiment(model, "This film is great")

0.9785954356193542